<a href="https://colab.research.google.com/github/RE-NY/Chatbot-with-nltk/blob/main/Chatbot_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install colorama

In [ ]:
import tensorflow
import nltk
import colorama
import numpy as np
import sklearn
import flask

### Data Preparation
now we will prepare(pre-process) our created data to train on our chatbot

In [ ]:
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Embedding , GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
#now we will read the created json file and process the required files......

with open("intents.json") as file:
  data = json.load(file)


training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    training_sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent["responses"])

  if intent["tag"] not in labels:
    labels.append(intent["tag"])

num_classes = len(labels)


In [ ]:
#now we will use LabelEncoder method...

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

### Tokenization
Now we will vectorize the data using tokenization which is an important step

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size , oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

### Training the neural network
Now we will train the architecture which extracts intentions from the sentences


In [ ]:
model = Sequential()
model.add(Embedding(vocab_size , embedding_dim , input_length = max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(units = 64 , activation = "relu"))
model.add(Dense(units = 128 , activation = "relu"))
model.add(Dense(units = 64 , activation = "relu"))
model.add(Dense(units = num_classes , activation = "softmax"))

model.compile(optimizer = "adam" , loss = "sparse_categorical_crossentropy" , metrics = ["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 64)                1088      
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 8)                 520       
                                                        

In [ ]:
epochs = 500
history = model.fit(padded_sequences , np.array(training_labels) , epochs = epochs)

Epoch 1/500
2/2 [==============================] - 1s 19ms/step - loss: 2.0798 - accuracy: 0.1515
Epoch 2/500
2/2 [==============================] - 0s 17ms/step - loss: 2.0770 - accuracy: 0.2121
Epoch 3/500
2/2 [==============================] - 0s 12ms/step - loss: 2.0757 - accuracy: 0.2121
Epoch 4/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0758 - accuracy: 0.2121
Epoch 5/500
2/2 [==============================] - 0s 11ms/step - loss: 2.0755 - accuracy: 0.2121
Epoch 6/500
2/2 [==============================] - 0s 12ms/step - loss: 2.0748 - accuracy: 0.2121
Epoch 7/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0739 - accuracy: 0.2121
Epoch 8/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0729 - accuracy: 0.1818
Epoch 9/500
2/2 [==============================] - 0s 11ms/step - loss: 2.0721 - accuracy: 0.1515
Epoch 10/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0709 - accuracy: 0.1515
Epoch 11/500
2/2 [=====

In [ ]:
#now lets save the model and fit of Tokenizer
model.save("chat_model")


import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#Now lets build chatbot with trained neural network......

#Now we will implement a chat function to interact with a real user. When the message from the user will be received,
#the chatbot will compute the similarity between the sequence of the new text and the training data.
#Taking into account the trust scores obtained for each category, it categorizes the user’s message according to an intention with the highest trust
# score

import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
      print(Fore.LIGHTBLACK_EX + "User : " + Style.RESET_ALL , end = "")
      inp = input()
      if inp.lower() == "quit":
        break
      result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
      tag = lbl_encoder.inverse_transform([np.argmax(result)])

      for i in data["intents"]:
        if i["tag"] == tag:
          print(Fore.LIGHTGREEN_EX + "Chatbot : " + Style.RESET_ALL , np.random.choice(i["responses"]))


print((Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL))
chat()


Start messaging with the bot (type quit to stop)!
User : Hello
1/1 [==============================] - 0s 153ms/step
Chatbot :  Hi
User : How are you?
1/1 [==============================] - 0s 24ms/step
Chatbot :  Have a nice day
User : age?
1/1 [==============================] - 0s 26ms/step
Chatbot :  Hello
User : What is your name?
1/1 [==============================] - 0s 26ms/step
Chatbot :  You can call me Joana.
User : oh nice name\
1/1 [==============================] - 0s 30ms/step
Chatbot :  Happy to help!
User : bye
1/1 [==============================] - 0s 22ms/step
Chatbot :  See you later
User : quit


In [ ]:
\
